In [38]:
# import numpy as np
import pickle

class Agent():
    
    def __init__(self,move_value):
        self.move_value = move_value
    
    def play(self):
        pass
    
    def update(self):
        pass

class Environment():
    
    def __init__(self):
        self.board = np.zeros((3,3))
    
    # Def: Check if there is any winner
    def check_winner(self):
        for row in self.board:
            if sum(row) == 3: return 1
            if sum(row) ==-3: return -1 
            
        for row in np.transpose(self.board):
            if sum(row) == 3: return 1
            if sum(row) ==-3: return -1 
            
        if sum([self.board[i][i] for i in range(len(self.board))]) == 3:  return 1
        if sum([self.board[i][i] for i in range(len(self.board))]) == -3: return -1
        if sum([self.board[i][len(self.board)-i-1] for i in range(len(self.board))]) == 3: return  1       
        if sum([self.board[i][len(self.board)-i-1] for i in range(len(self.board))]) == -3:return -1        
            
    # Def: Check if there is a winner
    def check_if_there_is_winner(self): return self.check_winner()
    
    # change the borad position - location and value.      
    def make_a_move(self,position,value):
        assert (value == 1 or value == -1)
        self.board.ravel()[position] = value
    
    # Reset the board back the starting position.     
    def reset_game(self): self.board = np.zeros((3,3))
        
    # View the current state of the board
    def get_board(self): return self.board

In [44]:
temp = Environment()
temp.make_a_move(0,1)
temp.make_a_move(1,1)
temp.make_a_move(8,-1)
print(temp.get_board())
print(temp.check_if_there_is_winner())
temp.make_a_move(2,1)
print(temp.get_board())
print(temp.check_if_there_is_winner())


[[ 1.  1.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -1.]]
None
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0. -1.]]
1


In [47]:
! git all-go

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean
